# Group Modeling Notebook

## Created by: Kim Buesser, Griffin Brown, and Kathryn O'Connor

# Introduction

Home Credit is an international finance provider aiming to increase financial inclusion by providing financial loans to those with little to no credit history. Rather than rely on credit history, Home Credit uses a variety of data to assess an individual’s repayment abilities.

While they currently use a variety of statistical and machine learning methods to make their predictions, Home Credit is hoping to further harness their data to ensure that those who are capable of repaying loans are not rejected. Achieving greater accuracy on whether an individual is capable of loan repayment will not only allow Home Credit to broaden their customer base (and in turn, profit) they will also be able to help a larger population of individuals who are in need of loans.

In this notebook, different models are explored with the aim of developing a model that beats the majority class classifier and can be used to help solve their business problem.

Processes such cross-validation and feature engineering are used with different models to help optimize overall performance, and metrics will be used to compare to the majority class classifier.

## Our Philosophy

In our project journey, we adopted a collaborative mindset. We agreed that keeping our data manipulations and dataset usage consistent across all models would streamline our process and maintain coherence. This unified approach not only enhances efficiency but also cultivates a harmonious environment conducive to reaching our mutual goals.

## Package Import

In [53]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt

## File Upload into Environment

Here, we stored our files in a Google Cloud Storage so we would not have to read directly from our local machines and change filepaths during development.

In [54]:
# Public URL after making the file public in the format 'https://storage.googleapis.com/...'
file_url = 'https://storage.googleapis.com/home_credit_files/application_train.csv'
test_url = 'https://storage.googleapis.com/home_credit_files/application_test.csv'
# POS_CASH_balance_url = 'https://storage.googleapis.com/home_credit_files/POS_CASH_balance.csv'
# bureau_url = 'https://storage.googleapis.com/home_credit_files/bureau.csv'
# bureau_balance_url = 'https://storage.googleapis.com/home_credit_files/bureau.csv'
# credit_card_balance = 'https://storage.googleapis.com/home_credit_files/credit_card_balance.csv'
# installments_payments = 'https://storage.googleapis.com/home_credit_files/credit_card_balance.csv'
# previous_application = 'https://storage.googleapis.com/home_credit_files/previous_application.csv'
sample_sub = 'https://storage.googleapis.com/home_credit_files/sample_submission.csv'


# Read the CSV directly from the URL
df = pd.read_csv(file_url)
test_df = pd.read_csv(test_url)

#print(df.head())

/var/folders/4l/0tw9tkvj2gx_v0qrnty9rt0r0000gn/T/ipykernel_30847/3497087744.py:14: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_url)


# EDA
Below, we perform EDA on the train data set to prepare it for modeling. We began by analyzing the initial data set and obtaining the proportion of the target variable in the train set. About 92% of the data is 0, indicating those that repayed their loans on time. About 8% represent 1, those that did not repay on time.

Next, we explored missing variables. We calculated the percentage of missing values per column, and after analyzing the descriptions for each, decided to remove those that were missing greater than 10% of their values.

In [55]:
print(df.shape)

(307511, 122)


In [56]:
df['TARGET'].value_counts()

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [57]:
df['TARGET'].value_counts(normalize = True) * 100

TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64

# Data Preparation

### Evaluating Missingness

In [58]:
# Train data set

# calculate total number of missing values for each column
missing_values_train = df.isnull().sum()

# calculate total number of rows
total_rows_train = df.shape[0]

# calculate percentage of missing values for each column
pct_missing_train = (missing_values_train / total_rows_train) * 100

# sort output
pct_missing_sorted_train = pct_missing_train.sort_values(ascending=False)

In [59]:
missings = pct_missing_sorted_train.to_frame(name='MissingPercentage')

### Removing columns with greater than 10% missing values

In [60]:
filter_missings = missings[(missings['MissingPercentage'] < 10)]
filter_missings.shape

(65, 1)

### Removing 'FLAG' columns and ID column

In [61]:
columns = list(filter_missings.index)
column_list = [item for item in columns if 'FLAG' not in item]

### Removing additional unhelpful columns

In [62]:
list1 = column_list
list2 = ['NAME_TYPE_SUITE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']

# Initialize an empty list to store items from list1 that are not in list2
items_not_in_list2 = []

# Iterate through each item in list1
for item in list1:
    # Check if the item is not in list2
    if item not in list2:
        # Add the item to the items_not_in_list2 list
        items_not_in_list2.append(item)


In [63]:
len(items_not_in_list2)

32

In [64]:
pd.set_option('display.max_columns', None)
selected_df = df[items_not_in_list2]
selected_df

test_filtered_columns = [item for item in items_not_in_list2 if item != 'TARGET']
test_selected_df = test_df[test_filtered_columns]

In [65]:
# Extract the 'TARGET' column
target = selected_df['TARGET']

# Remove the 'TARGET' column from the dataframe and reassign the result back to selected_df
selected_df = selected_df.drop(columns=['TARGET'])

# Insert the 'TARGET' column at the beginning of the dataframe
selected_df.insert(0, 'TARGET', target)

### Fix problematic values

In [66]:
selected_df.loc[:, 'DAYS_EMPLOYED'] = selected_df['DAYS_EMPLOYED'].replace(365243, 0)
selected_df = selected_df[selected_df['AMT_INCOME_TOTAL'] <= 9000000]

test_selected_df.loc[:, 'DAYS_EMPLOYED'] = test_selected_df['DAYS_EMPLOYED'].replace(365243, 0)
test_selected_df = test_selected_df[test_selected_df['AMT_INCOME_TOTAL'] <= 9000000]

In [67]:
print(selected_df.shape)
print(test_selected_df.shape)

(307508, 32)
(48744, 31)


In [68]:
selected_df = selected_df.drop(columns = ['HOUR_APPR_PROCESS_START', 'DAYS_ID_PUBLISH'])
test_selected_df = test_selected_df.drop(columns = ['HOUR_APPR_PROCESS_START', 'DAYS_ID_PUBLISH'])

In [69]:
set(test_selected_df)-set(selected_df)

set()

In [70]:
# Abs value of negatives
selected_df['DAYS_LAST_PHONE_CHANGE'] = selected_df.loc[:, 'DAYS_LAST_PHONE_CHANGE'].abs()
selected_df['DAYS_REGISTRATION'] = selected_df.loc[:, 'DAYS_REGISTRATION'].abs()
selected_df['DAYS_EMPLOYED'] = selected_df.loc[:, 'DAYS_EMPLOYED'].abs()
selected_df['DAYS_BIRTH'] = selected_df.loc[:, 'DAYS_BIRTH'].abs()

test_selected_df['DAYS_LAST_PHONE_CHANGE'] = test_selected_df.loc[:, 'DAYS_LAST_PHONE_CHANGE'].abs()
test_selected_df['DAYS_REGISTRATION'] = test_selected_df.loc[:, 'DAYS_REGISTRATION'].abs()
test_selected_df['DAYS_EMPLOYED'] = test_selected_df.loc[:, 'DAYS_EMPLOYED'].abs()
test_selected_df['DAYS_BIRTH'] = test_selected_df.loc[:, 'DAYS_BIRTH'].abs()

## Data Preparation Summary

Our initial approach was fairly simplistic for our first attempt. We started by dropped columns that had too much missing data with a cutoff of 10%. We then cut out all 'FLAG' columns as we thought these weren't needed for our basic approach. We fixed a couple columns that had some bad information, and all the while we did the same manipulations to the test set as we did with the train set. 

## Preparing Logistic Regression

In [71]:
# Grab numeric columns
numeric_cols = selected_df.drop(['SK_ID_CURR'], axis=1).select_dtypes(include='number')
numeric_cols

test_numeric_cols = test_selected_df.drop(['SK_ID_CURR'], axis=1).select_dtypes(include='number')
test_numeric_cols

,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,AMT_CREDIT,AMT_INCOME_TOTAL,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE
0,0.789654,450000.0,20560.5,2.0,1740.0,0,568800.0,135000.0,0,0,0,0,0,0,2,2,5170.0,2329,19241,0.018850
1,0.291656,180000.0,17370.0,2.0,0.0,0,222768.0,99000.0,0,0,0,0,0,0,2,2,9118.0,4469,18064,0.035792
2,0.699787,630000.0,69777.0,2.0,856.0,0,663264.0,202500.0,0,0,0,0,0,0,2,2,2175.0,4458,20038,0.019101
3,0.509677,1575000.0,49018.5,4.0,1805.0,2,1575000.0,315000.0,0,0,0,0,0,0,2,2,2000.0,1866,13976,0.026392
4,0.425687,625500.0,32067.0,3.0,821.0,1,625500.0,180000.0,1,1,0,0,0,0,2,2,4000.0,2191,13040,0.010032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.648575,270000.0,17473.5,1.0,684.0,0,412560.0,121500.0,0,0,0,0,0,0,3,3,9094.0,5169,19970,0.002042
48740,0.684596,495000.0,31909.5,4.0,0.0,2,622413.0,157500.0,1,1,0,0,0,0,2,2,3015.0,1149,11186,0.035792
48741,0.632770,315000.0,33205.5,3.0,838.0,1,315000.0,202500.0,0,0,0,0,0,0,2,2,2681.0,3037,15922,0.026392
48742,0.445701,450000.0,25128.0,2.0,2308.0,0,450000.0,225000.0,1,1,0,1,1,0,2,2,1461.0,2731,13968,0.018850


In [72]:
# Create a list of numeric column names excluding target for imputations

column_names = numeric_cols.columns.tolist()
my_list = [x for x in column_names if x != 'TARGET']
print(my_list)

test_column_names = numeric_cols.columns.tolist()
test_my_list = [x for x in column_names if x != 'TARGET']
print(test_my_list)

['EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE', 'CNT_CHILDREN', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'LIVE_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_POPULATION_RELATIVE']
['EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE', 'CNT_CHILDREN', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'LIVE_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_POPULATION_RELATIVE']


### N/a Imputer

In [73]:
# Create a SimpleImputer instance
imputer = SimpleImputer(strategy='median')

# Fit and transform the selected numeric columns
selected_df.loc[:, my_list] = imputer.fit_transform(selected_df.loc[:, my_list])

test_selected_df.loc[:, test_my_list] = imputer.fit_transform(test_selected_df.loc[:, test_my_list])

## Interaction Term

In [74]:
selected_df['CREDIT_TO_INCOME'] = selected_df['AMT_CREDIT']/selected_df['AMT_INCOME_TOTAL']

test_selected_df['CREDIT_TO_INCOME'] = test_selected_df['AMT_CREDIT']/test_selected_df['AMT_INCOME_TOTAL']

### Standard Scaler to scale dataset

In [75]:
#scalar_list = ['EXT_SOURCE_2','AMT_GOODS_PRICE','AMT_ANNUITY','AMT_CREDIT','AMT_INCOME_TOTAL','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','CREDIT_TO_INCOME']

In [76]:
scaler = StandardScaler()

# Fit and transform the selected columns
selected_df[my_list] = scaler.fit_transform(selected_df[my_list])

test_selected_df[test_my_list] = scaler.fit_transform(test_selected_df[test_my_list])

print(selected_df.shape)
print(test_selected_df.shape)

(307508, 31)
(48744, 30)


### Dummy Encoder 

In [77]:
# dummy encoding data set

selected_df = pd.get_dummies(selected_df, drop_first=True)

test_selected_df = pd.get_dummies(test_selected_df, drop_first=True)

print(selected_df.shape)
print(test_selected_df.shape)

print(set(test_selected_df)-set(selected_df))
print(set(selected_df)-set(test_selected_df))

columns_to_drop = set(selected_df.columns) - set(test_selected_df.columns)
columns_to_drop.discard('TARGET')
selected_df = selected_df.drop(columns=list(columns_to_drop), axis=1)

print(selected_df.shape)
print(test_selected_df.shape)

(307508, 110)
(48744, 106)
set()
{'TARGET', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown', 'NAME_INCOME_TYPE_Maternity leave'}
(307508, 107)
(48744, 106)


### Splitting Target and Predictors

In [78]:
X = selected_df.drop(columns=['TARGET', 'SK_ID_CURR'])
y = selected_df['TARGET']

## Upsampling

In [90]:
X = selected_df.drop('TARGET', axis=1)
y = selected_df['TARGET']

data = pd.concat([X, y], axis=1)

# Separate majority and minority classes
majority = data[data.TARGET==0]
minority = data[data.TARGET==1]

# Upsample minority class
minority_upsampled = resample(minority, 
                              replace=True,     # sample with replacement
                              n_samples=len(majority),    # to match majority class size
                              random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
upsampled_data = pd.concat([majority, minority_upsampled])

# Checking the new class distribution
upsampled_data.TARGET.value_counts()

X_upsampled = upsampled_data.drop('TARGET', axis=1)
y_upsampled = upsampled_data['TARGET']


## Logistic Regression 

In [91]:
# Initializing and training the logistic regression model
log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X, y)

# Making predictions on the validation set
y_pred = log_reg.predict(X)

# Calculating the accuracy of the model
accuracy = accuracy_score(y, y_pred)
print("Train Accuracy:", accuracy)
print("Train R2 = ", log_reg.score(X, y))

Train Accuracy: 0.9192736449133031
Train R2 =  0.9192736449133031


In [92]:
# Predict probabilities for the positive class (class 1) using the trained model
y_prob = log_reg.predict_proba(X)[:, 1]

# Calculate false positive rate, true positive rate, and thresholds
fpr, tpr, thresholds = roc_curve(y, y_prob)

# Calculate AUC score
auc_score = auc(fpr, tpr)
print("AUC Score (Original):", auc_score)

AUC Score (Original): 0.5022168461434817


## Logistic Regression Upsampled

In [93]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Upsample the dataset
X_upsampled, y_upsampled = smote.fit_resample(X_upsampled, y_upsampled)

# Initialize and train the logistic regression model with the upsampled dataset
log_reg_upsampled = LogisticRegression(max_iter=2000)
log_reg_upsampled.fit(X_upsampled, y_upsampled)

# Make predictions on the upsampled dataset
y_pred_upsampled = log_reg_upsampled.predict(X_upsampled)

# Calculate accuracy on the upsampled dataset
accuracy_upsampled = accuracy_score(y_upsampled, y_pred_upsampled)
print("Train Accuracy (Upsampled):", accuracy_upsampled)
print("Train R2 (Upsampled):", log_reg_upsampled.score(X_upsampled, y_upsampled))

Train Accuracy (Upsampled): 0.5
Train R2 (Upsampled): 0.5


In [94]:
# Predict probabilities for the positive class (class 1) using upsampled model
y_prob_upsampled = log_reg_upsampled.predict_proba(X_upsampled)[:, 1]

# Calculate false positive rate, true positive rate, and thresholds for upsampled model
fpr_upsampled, tpr_upsampled, thresholds_upsampled = roc_curve(y_upsampled, y_prob_upsampled)

# Calculate AUC score for upsampled model
auc_score_upsampled = auc(fpr_upsampled, tpr_upsampled)
print("AUC Score (Upsampled):", auc_score_upsampled)

AUC Score (Upsampled): 0.5032005156743218


## Cross Validation

### Mean Squared Error - simple logistic model

In [95]:


# Call the function  and define a scoring function - set it as "neg_mean_squared_error"
log_scores = cross_val_score(log_reg, 
                             X, 
                             y,
                             scoring="neg_mean_squared_error",   # higher return values are better for this function
                             cv=10                               # create 10 folds
)

# to obtain RMSE, we need to negate the scores obtained  from cross_val_score and take the sqrt.
# this returns an array of 10 values
log_rmse_scores = np.sqrt(-log_scores)


In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
display_scores(log_rmse_scores)

### Mean Squared Error - upsampled logistic model

In [ ]:
log_scores_upsampled = cross_val_score(log_reg_upsampled, 
                                       X_upsampled, 
                                       y_upsampled,
                                       scoring="neg_mean_squared_error",   # higher return values are better for this function
                                       cv=10 # create 10 folds
)

log_rmse_scores_upsampled = np.sqrt(-log_scores_upsampled)

In [96]:
display_scores(log_rmse_scores_upsampled)

Scores: [0.6295993  0.70710053 0.70710053 0.70711303 0.70711303 0.54084786
 0.50124983 0.70711303 0.70710678 0.70710678]
Mean: 0.6621450712088042
Standard deviation: 0.0747067396895937


### R Squared - simple logistic model

In [97]:
log_r2_scores = cross_val_score(log_reg, 
                             X, y,
                             scoring="r2", 
                             cv=10)
display_scores(log_r2_scores)

Scores: [-0.08779936 -0.08779936 -0.08779936 -0.08779936 -0.08783784 -0.08783784
 -0.08783784 -0.08783784 -0.08780246 -0.08780246]
Mean: -0.08781537003882575
Standard deviation: 1.837990557490588e-05


In [98]:
log_auc_scores = cross_val_score(log_reg, 
                             X, 
                             y,
                             scoring="roc_auc", 
                             cv=10                               
)
log_mean_auc = np.mean(log_auc_scores)
display_scores(log_auc_scores)

Scores: [0.55891164 0.47320167 0.77771346 0.84179435 0.23103222 0.68945731
 0.5061176  0.51077052 0.49801962 0.47524805]
Mean: 0.5562266429015005
Standard deviation: 0.1659910446997831


### R squared - upsampled logistic model

In [99]:
log_up_r2_scores = cross_val_score(log_reg_upsampled, 
                             X_upsampled, 
                             y_upsampled,
                             scoring="r2", 
                             cv=10)
display_scores(log_up_r2_scores)

Scores: [-1.00003538 -0.99996463 -0.99996463 -1.00003538 -1.00003538 -1.00003538
 -0.99996463 -1.00003538 -1.         -1.        ]
Mean: -1.000007075513384
Standard deviation: 3.0839294641349914e-05


In [82]:
log_up_auc_scores = cross_val_score(log_reg_upsampled, 
                             X_upsampled, 
                             y_upsampled,
                             scoring="roc_auc", 
                             cv=10                               
)
log_up_mean_auc = np.mean(log_up_auc_scores)
display_scores(log_up_auc_scores)

Scores: [0.39590006 0.37154273 0.26070667 0.10165558 0.47117858 0.50665821
 0.39784145 0.36870688 0.30433986 0.25336769]
Mean: 0.3431897719134749
Standard deviation: 0.11182254153778533


## XGboost

In [100]:

xgb_clf = XGBClassifier(use_label_encoder=False, 
                        eval_metric='logloss', 
                        n_estimators=100, 
                        max_depth=15
                        )

xgb_clf.fit(X, y)

xgb_y_pred = xgb_clf.predict(X)

xgb_accuracy = accuracy_score(y, xgb_y_pred)

print("Train Accuracy:", xgb_accuracy)
print("Train Accuracy using built-in score:", xgb_clf.score(X, y))

/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Train Accuracy: 0.9906604055829442
Train Accuracy using built-in score: 0.9906604055829442


In [ ]:
xgb_auc_scores = cross_val_score(xgb_clf, 
                             X, 
                             y,
                             scoring="roc_auc", 
                             cv=10                               
)
xgb_mean_auc = np.mean(xgb_auc_scores)
display_scores(xgb_auc_scores)

/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/kathrynoconnor/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encod

In [ ]:
xgb_cm = confusion_matrix(y, xgb_y_pred)

# Visualizing the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(xgb_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for XGBoost Classifier')
plt.show()


## XGBoost Upsample

In [ ]:
xgb_clf_upsampled = XGBClassifier(use_label_encoder=False, 
                                  eval_metric='logloss', 
                                  n_estimators=100, 
                                  max_depth=6)

# Train the model on the upsampled dataset
xgb_clf_upsampled.fit(X_upsampled, y_upsampled)

# Make predictions (as an example, using the same upsampled data for simplicity)
y_pred_upsampled = xgb_clf_upsampled.predict(X_upsampled)

# Calculate accuracy
accuracy_upsampled = accuracy_score(y_upsampled, y_pred_upsampled)
print("Train Accuracy on Upsampled Data:", accuracy_upsampled)

In [ ]:
xgb_upsampled_cm = confusion_matrix(y, y_pred_upsampled)

# Visualizing the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(xgb_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for XGBoost Classifier')
plt.show()

## Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, 
                                max_depth=4, 
                                random_state=42)

rf_clf.fit(X, y)

rf_y_pred = rf_clf.predict(X)

accuracy = accuracy_score(y, rf_y_pred)
print("Train Accuracy:", accuracy)
print("Train Accuracy using built-in score:", rf_clf.score(X, y))

In [ ]:
rf_auc_scores = cross_val_score(rf_clf, 
                             X, 
                             y,
                             scoring="roc_auc", 
                             cv=10                               
)
rf_mean_auc = np.mean(rf_auc_scores)
display_scores(rf_auc_scores)

In [ ]:
rf_cm = confusion_matrix(y, rf_y_pred)

# Visualizing the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(rf_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Random Forest Classifier')
plt.show()

In [ ]:
single_tree = rf_clf.estimators_[0]

plt.figure(figsize=(20,10))
plot_tree(single_tree, filled=True, feature_names=X.columns, class_names=True, rounded=True)
plt.show()

## Random Forest Upsampled

In [ ]:
rf_clf_upsampled = RandomForestClassifier(n_estimators=100, 
                                          max_depth=4, 
                                          random_state=42)

rf_clf_upsampled.fit(X_upsampled, y_upsampled)

# Making predictions on the upsampled training set for demonstration
rf_y_pred_upsampled = rf_clf_upsampled.predict(X_upsampled)

# Calculate accuracy
accuracy_upsampled = accuracy_score(y_upsampled, rf_y_pred_upsampled)
print("Train Accuracy on Upsampled Data:", accuracy_upsampled)

# Test Evaluation

In [ ]:
# Step 1: Assuming 'test_df' contains 'SK_ID_CURR' and you're about to predict
# Save the IDs in a separate variable or ensure they're in 'test_df'
ids = test_selected_df['SK_ID_CURR'].copy()

# If you had to remove 'SK_ID_CURR' from test_df for prediction, do so
test_features = test_selected_df.drop(columns=['SK_ID_CURR'])

# Step 2: Predict TARGET on the test set
xgb_y_test_pred = xgb_clf_upsampled.predict(test_features)

# Step 3: Combine IDs with their corresponding predictions
predictions_with_ids = pd.DataFrame({'SK_ID_CURR': ids, 'TARGET': xgb_y_test_pred})

print(predictions_with_ids['TARGET'].value_counts())

# Now you can see the dataset with IDs and the new TARGET column
print(predictions_with_ids.head())
sample = pd.read_csv(sample_sub)
print(sample.head())


In [ ]:
directory = 'C:\\Users\\Owner\\OneDrive\\Documents'
filename = 'predictions_with_ids.csv'
path = os.path.join(directory, filename)

# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Now it's safe to write the file
predictions_with_ids.to_csv(path, index=False)

## Results

During our exploratory data analysis (EDA) and initial model building, we identified two key challenges: the imbalanced distribution of the target variable and the presence of a substantial number of missing values. Addressing these challenges was crucial for enhancing our model performance.

Upsampling emerged as a pivotal technique, significantly elevating the Area Under the Curve (AUC) scores across all our models. This outcome was particularly notable due to the imbalance in the target variable, with over 91% of outcomes being 0. By incorporating upsampling, our models were trained on a more balanced dataset, allowing them to capture the minority class (1) more effectively.

The application of upsampling directly aligns with Home Credit's strategic objective of minimizing loan rejections for creditworthy individuals. We believe that leveraging techniques like upsampling in predictive modeling is instrumental in achieving this goal.

Furthermore, we recommend exploring additional avenues such as interaction variables and feature engineering to further optimize predictive performance. These strategies can potentially unlock deeper insights from the data and enhance the robustness of our models.